In [2]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')


# 0) 관련 모듈 임포트 
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 공공데이터활용지원센터_보건복지부 코로나19 감염 현황
- 코로나19감염증으로 인한 일별 확진자, 완치자, 치료중인환자, 사망자등에 대한 현황자료
- https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043376
- [활용신청] 클릭후 서비스키 정보 확인 [마이페이지]-[오픈 API]-[개발 계정]

일반 인증키 (Encoding)                
W7qHt8KLRZawPAd%2BiTUo1nt2EcSMgHwbxxMGhnKn3wfwLcwrVnHHmaVbPIi%2BgecqaouM%2ByjaLydMoXwiJm%2BMjQ%3D%3D

일반 인증키(Decoding)    
W7qHt8KLRZawPAd+iTUo1nt2EcSMgHwbxxMGhnKn3wfwLcwrVnHHmaVbPIi+gecqaouM+yjaLydMoXwiJm+MjQ==

In [17]:
# 2) API 신청 => xml 텍스트 데이타

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
serviceKey = 'Yhq6n9vlpaKZsCOw0jec8zIdZ8p+Bpuku6WLjVgDhqXRW6dHnfXoauSEj19jpnjv59CUGspyTxVCgCeXOua7dg=='
# params ={'serviceKey' : '서비스키', 'pageNo' : '1', 'numOfRows' : '10', 'startCreateDt' : '20200310', 'endCreateDt' : '20200414' }
params ={'serviceKey' : serviceKey,
         'pageNo' : '1', 
         'numOfRows' : '10', 
         'startCreateDt' : '20220515', 
         'endCreateDt' : '20220531' }

response = requests.get(url, params=params)
# print(response.content)
xml_str = response.text
# xml_str
soup = BeautifulSoup(xml_str, 'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><createDt>2022-05-31 08:55:39.977</createDt><deathCnt>24176</deathCnt><decideCnt>18103577</decideCnt><seq>896</seq><stateDt>20220531</stateDt><stateTime>00:00</stateTime><updateDt>2022-06-04 08:59:23.637</updateDt></item><item><createDt>2022-05-30 09:01:30.847</createDt><deathCnt>24167</deathCnt><decideCnt>18086392</decideCnt><seq>895</seq><stateDt>20220530</stateDt><stateTime>00:00</stateTime><updateDt>2022-06-04 08:59:37.203</updateDt></item><item><createDt>2022-05-29 09:05:28.382</createDt><deathCnt>24158</deathCnt><decideCnt>18080257</decideCnt><seq>894</seq><stateDt>20220529</stateDt><stateTime>00:00</stateTime><updateDt>2022-06-04 08:59:51.863</updateDt></item><item><createDt>2022-05-28 09:09:18.909</createDt><deathCnt>24139</deathCnt><decideCnt>18067610</decideCnt><seq>893</seq><stateDt>20220528</stateDt><stateTime>00:00</stateTime

In [19]:
# 3) 소스파악 및 타겟리스트 설정
# items > item ...
target_list = soup.find_all('item')
len(target_list)



17

In [20]:
target_list[0]

<item><createDt>2022-05-31 08:55:39.977</createDt><deathCnt>24176</deathCnt><decideCnt>18103577</decideCnt><seq>896</seq><stateDt>20220531</stateDt><stateTime>00:00</stateTime><updateDt>2022-06-04 08:59:23.637</updateDt></item>

In [21]:
target_list[-1]

<item><createDt>2022-05-15 09:27:15.538</createDt><deathCnt>23709</deathCnt><decideCnt>17781952</decideCnt><seq>880</seq><stateDt>20220515</stateDt><stateTime>00:00</stateTime><updateDt>2022-05-21 08:47:54.484</updateDt></item>

In [29]:
# 4) 2차원 리스트 => 데이타프레임
# 등록일시분초 createdt
# 사망자 수 deathCnt
# 확진자 수 decideCnt
# 기준일 statedt
# 기준시간 statetime
# 수정일시분초 updateDt 

covid_list = []
for item in target_list:
    createDt = item.createDt.text
    deathCnt = item.deathCnt.text
    decideCnt = item.decideCnt.text
    print(createDt, deathCnt, decideCnt)
    covid_list.append([createDt, deathCnt, decideCnt])
df = pd.DataFrame(covid_list, columns=['등록날짜', '사망자수', '확진자수'])
df

2022-05-31 08:55:39.977 24176 18103577
2022-05-30 09:01:30.847 24167 18086392
2022-05-29 09:05:28.382 24158 18080257
2022-05-28 09:09:18.909 24139 18067610
2022-05-27 09:03:34.444 24103 18053236
2022-05-26 09:13:37.732 24063 18036664
2022-05-25 09:02:44.294 24029 18017859
2022-05-24 08:49:38.261 24006 17993923
2022-05-23 08:56:56.412 23987 17967592
2022-05-22 07:52:40.527 23965 17957621
2022-05-21 08:39:51.786 23911 17938332
2022-05-20 09:02:54.987 23885 17914882
2022-05-19 09:07:11.841 23842 17889769
2022-05-18 08:58:30.377 23802 17861660
2022-05-17 09:04:52.287 23771 17830327
2022-05-16 09:04:43.174 23744 17795242
2022-05-15 09:27:15.538 23709 17781952


,등록날짜,사망자수,확진자수
0,2022-05-31 08:55:39.977,24176,18103577
1,2022-05-30 09:01:30.847,24167,18086392
2,2022-05-29 09:05:28.382,24158,18080257
3,2022-05-28 09:09:18.909,24139,18067610
4,2022-05-27 09:03:34.444,24103,18053236
5,2022-05-26 09:13:37.732,24063,18036664
6,2022-05-25 09:02:44.294,24029,18017859
7,2022-05-24 08:49:38.261,24006,17993923
8,2022-05-23 08:56:56.412,23987,17967592
9,2022-05-22 07:52:40.527,23965,17957621
